In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
from entmax import sparsemax  # Requires: pip install entmax


In [23]:
class PortfolioOptimization(nn.Module):
    def __init__(self, num_assets, num_features=4, hidden_dim=128, input_seq_len=10):
        super(PortfolioOptimization, self).__init__()
        self.num_assets = num_assets
        self.hidden_dim = hidden_dim
        self.input_seq_len = input_seq_len
        
        # Feature extraction
        self.dcc1 = DCC(num_features, hidden_dim, kernel_size=3, dilation=1)
        self.dcc2 = DCC(hidden_dim, hidden_dim, kernel_size=3, dilation=2)
        self.dcc3 = DCC(hidden_dim, hidden_dim, kernel_size=3, dilation=4)

        # Attention mechanism
        self.Wq = nn.Linear(num_features * input_seq_len, hidden_dim)
        self.Wk = nn.Linear(num_features * input_seq_len, hidden_dim)
        self.Wv = nn.Linear(num_features * input_seq_len, hidden_dim)

        # self.Wq = nn.Linear(40, hidden_dim)
        # self.Wk = nn.Linear(40, hidden_dim)
        # self.Wv = nn.Linear(40, hidden_dim)

        # Graph attention
        self.gat = GATLayer(hidden_dim, hidden_dim)

        # Feature combination
        self.Wr = nn.Linear(hidden_dim, hidden_dim)
        self.We = nn.Linear(hidden_dim, hidden_dim)

        # Prediction head
        self.dcc_pred = DCC(hidden_dim * 3, num_features, kernel_size=3, dilation=1)

        # Policy head
        self.Wf = nn.Linear(hidden_dim * 3, hidden_dim)
        self.conv_policy = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=1)
        self.Wt = nn.Linear(hidden_dim + num_features, hidden_dim)
        self.Ww = nn.Linear(hidden_dim, num_assets)

        # Allocation parameters
        self.min_weight = 0.05
        self.rank_power = 1.0
        self.temperature = nn.Parameter(torch.tensor(1.0))  # Learnable temperature
        self.price_scale = nn.Parameter(torch.ones(num_features))
        self.price_bias = nn.Parameter(torch.zeros(num_features))

    def forward(self, x, prev_weights=None):
       
        if x.dim() == 3:
            x = x.unsqueeze(0)
        batch_size, seq_len, num_assets, num_features = x.size()
    
        # Feature extraction
        x_reshaped = x.permute(0, 2, 3, 1).reshape(batch_size * num_assets, num_features, seq_len)
        fe = self.dcc1(x_reshaped)
        fe = self.dcc2(fe)
        fe = self.dcc3(fe)
        fe = fe[:, :, -1].reshape(batch_size, num_assets, self.hidden_dim)
    
        # Cross-asset attention
        patches_flat = x.reshape(batch_size, num_assets, -1)
        q = self.Wq(patches_flat)
        k = self.Wk(patches_flat)
        v = self.Wv(patches_flat)
        attention = torch.bmm(q, k.transpose(1, 2)) / (self.hidden_dim ** 0.5)
        attention = F.softmax(attention, dim=-1)
    
        # Graph attention
        fr = self.gat(fe, attention)
    
        # Market context
        fm = self.Wr(fr) + self.We(fe)
        fm = fm.mean(dim=1, keepdim=True).expand(-1, num_assets, -1)
    
        # Combined features
        f = torch.cat([fe, fr, fm], dim=-1)
        f_reshaped = f.permute(0, 2, 1)
        x_pred = self.dcc_pred(f_reshaped)
        x_pred = x_pred.permute(0, 2, 1)
        
        # Ensure positive predictions using modified softplus
        x_pred = F.softplus(x_pred) * 1.5  # Scale to prevent predictions from being too small
    
        # Policy features
        f_policy = F.relu(self.conv_policy(self.Wf(f).permute(0, 2, 1))).permute(0, 2, 1)
        f_policy = self.Wt(torch.cat([f_policy, x_pred], dim=-1))
        
        if prev_weights is not None:
            f_policy = f_policy + prev_weights.unsqueeze(-1)
    
        # Raw score for allocation
        raw_scores = self.Ww(f_policy[:, -1])  # shape: [B, A]
        
        # Get predicted returns from the Close prices (index 1 in features)
        predicted_returns = x_pred[:, :, 1]  # shape: [B, A]
        
        # Combine raw scores with predicted returns for ranking
        combined_scores = raw_scores + predicted_returns.detach()  # Detach to prevent double counting
        
        # Create differentiable rank weights
        ranks = torch.argsort(torch.argsort(combined_scores, dim=1, descending=True))
        rank_weights = 1.0 / (ranks.float() + 1)  # 1/rank weighting
        
        # Normalize rank weights
        rank_weights = rank_weights / rank_weights.sum(dim=1, keepdim=True)
        
        # Apply temperature-scaled sparsemax to combined scores
        scaled_scores = combined_scores / (self.temperature + 1e-8)
        sparse_weights = sparsemax(scaled_scores, dim=-1)
        
        # Blend sparse weights with rank weights (adjust ratio as needed)
        weights = 0.7 * sparse_weights + 0.3 * rank_weights
        
        # Apply min allocation constraint and normalize
        weights = self.min_weight + (1.0 - self.min_weight * num_assets) * weights
        weights = weights / weights.sum(dim=1, keepdim=True)
        
        return x_pred, weights

class DCC(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super(DCC, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size,
                              padding=(kernel_size - 1) * dilation, dilation=dilation)
        self.kernel_size = kernel_size
        self.dilation = dilation

    def forward(self, x):
        x = self.conv(x)
        padding = (self.kernel_size - 1) * self.dilation
        x = x[:, :, :-padding] if padding > 0 else x
        return F.relu(x)


class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.1):
        super(GATLayer, self).__init__()
        self.Wg = nn.Linear(in_features, out_features, bias=False)
        self.Wa = nn.Linear(2 * in_features, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, node_features, edge_info):
        batch_size, num_nodes, in_features = node_features.size()
        h = self.Wg(node_features)

        attention = torch.zeros(batch_size, num_nodes, num_nodes, device=node_features.device)
        for i in range(num_nodes):
            for j in range(num_nodes):
                concat_features = torch.cat([node_features[:, i, :], node_features[:, j, :]], dim=-1)
                gate = torch.tanh(self.Wa(concat_features))
                attention[:, i, j] = edge_info[:, i, j] * gate.squeeze()

        attention = F.softmax(attention, dim=-1)
        attention = self.dropout(attention)

        h_prime = torch.bmm(attention, h)
        return F.elu(h_prime)



In [24]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def predict_with_reallocation(model, initial_sequence, initial_weights, num_days=20, initial_capital=10000):
    model.eval()
    asset_names = ['ONGC', 'BPCL', 'HINDPETRO', 'NMDC', 'IRFC', 'IOC']  # Modify as per actual order
    
    current_sequence = initial_sequence.clone().to(device)  # Shape: [10, A, 4]
    current_weights = torch.tensor(initial_weights, dtype=torch.float32, device=device)
    capital = initial_capital
    allocations = capital * current_weights.cpu().numpy()

    predicted_prices = []
    weight_history = []
    transaction_history = []
    portfolio_values = []

    dates = [datetime.today() + timedelta(days=i) for i in range(num_days)]

    for day in range(num_days):
        with torch.no_grad():
            x_input = current_sequence.unsqueeze(0)  # [1, 10, A, 4]
            x_pred, new_weights = model(x_input, prev_weights=current_weights.unsqueeze(0))  # [1, A, 4], [1, A]

            # Extract predicted features for current day
            predicted_day = x_pred[0].cpu().numpy()
            predicted_prices.append(predicted_day)

            # Use Close price (index 1)
            predicted_close = predicted_day[:, 1]

            if day > 0:
                # Compute return over previous day
                prev_close = current_sequence[-1, :, 1].cpu().numpy()
                allocations *= (predicted_close / prev_close)
                capital = allocations.sum()
            
            # Rebalancing
            new_weights_np = new_weights[0].cpu().numpy()
            target_allocations = new_weights_np * capital
            transactions = target_allocations - allocations
            allocations = target_allocations
            current_weights = new_weights[0]
            portfolio_values.append(capital)

            # Update sequence with predicted day
            predicted_tensor = torch.from_numpy(predicted_day).unsqueeze(0).to(device)  # [1, A, 4]
            current_sequence = torch.cat([current_sequence[1:], predicted_tensor], dim=0)

            # Log
            weight_history.append(new_weights_np)
            transaction_history.append(transactions)

    # DataFrames
    pred_df = pd.DataFrame(np.array(predicted_prices)[:, :, 1], index=dates, columns=asset_names)
    weight_df = pd.DataFrame(np.array(weight_history), index=dates, columns=asset_names)
    transaction_df = pd.DataFrame(np.array(transaction_history), index=dates, columns=asset_names)
    value_df = pd.DataFrame({'Portfolio Value': portfolio_values}, index=dates)

    return pred_df, weight_df, transaction_df, value_df


In [25]:
def save_and_plot(pred_df, weight_df, transaction_df, value_df):
    # Save to Excel
    with pd.ExcelWriter("portfolio_reallocation_result.xlsx") as writer:
        pred_df.to_excel(writer, sheet_name="Predicted Close")
        weight_df.to_excel(writer, sheet_name="Weights")
        transaction_df.to_excel(writer, sheet_name="Transactions")
        value_df.to_excel(writer, sheet_name="Portfolio Value")

    # Plot
    plt.figure(figsize=(16, 10))

    # Portfolio Value
    plt.subplot(2, 2, 1)
    plt.plot(value_df.index, value_df['Portfolio Value'], color='black')
    plt.title("Portfolio Value Over Time")
    plt.ylabel("₹ Value")

    # Predicted Prices
    plt.subplot(2, 2, 2)
    for asset in pred_df.columns:
        plt.plot(pred_df.index, pred_df[asset], label=asset)
    plt.title("Predicted Close Prices")
    plt.legend()

    # Weights
    plt.subplot(2, 2, 3)
    for asset in weight_df.columns:
        plt.plot(weight_df.index, weight_df[asset], label=asset)
    plt.title("Portfolio Weights")
    plt.legend()

    # Rebalancing
    plt.subplot(2, 2, 4)
    for asset in transaction_df.columns:
        plt.plot(transaction_df.index, transaction_df[asset], label=asset)
    plt.title("Rebalancing Transactions (₹)")
    plt.legend()

    plt.tight_layout()
    plt.show()


In [30]:
def predict_with_reallocation(model, initial_sequence, initial_weights, num_days=20, initial_capital=10000):
    """
    Predict prices and dynamically rebalance portfolio over time
    Args:
        model: Trained PortfolioOptimization model
        initial_sequence: Last available sequence [seq_len, num_assets, num_features]
        initial_weights: Starting weight distribution [num_assets]
        num_days: Prediction horizon
        initial_capital: Starting investment amount
    Returns:
        pred_prices: Predicted prices [num_days, num_assets, num_features]
        weight_history: Weight evolution [num_days, num_assets]
        transaction_history: Daily transactions [num_days, num_assets]
    """
    model.eval()
    assets = ['ONGC', 'BPCL', 'HINDPETRO', 'NMDC', 'IRFC', 'IOC']
    
    # Initialize tracking
    current_sequence = initial_sequence.clone().to(device)
    current_weights = torch.tensor(initial_weights, dtype=torch.float32, device=device)
    capital = initial_capital
    allocations = capital * current_weights.cpu().numpy()
    
    pred_prices = []
    weight_history = []
    transaction_history = []
    dates = [datetime.today() + timedelta(days=i) for i in range(num_days)]
    
    for day in range(num_days):
        with torch.no_grad():
            # Predict next prices and optimal weights
            x_input = current_sequence.unsqueeze(0)  # Add batch dimension
            x_pred, new_weights = model(x_input, current_weights.unsqueeze(0))
            
            # Store predictions (use all features: Open, Close, High, Low)
            day_pred = x_pred[0].cpu().numpy()
            pred_prices.append(day_pred)
            
            # Calculate current portfolio value
            if day > 0:
                prev_close = current_sequence[-1, :, 1].cpu().numpy()  # Previous close prices
                current_close = day_pred[:, 1]  # Predicted close prices
                allocations *= (current_close / prev_close)
                capital = allocations.sum()
            
            # Calculate rebalancing transactions (in monetary terms)
            target_alloc = new_weights[0].cpu().numpy() * capital
            transactions = target_alloc - allocations
            allocations = target_alloc  # Update to new allocations
            
            # Record results
            weight_history.append(new_weights[0].cpu().numpy())
            transaction_history.append(transactions)
            
            # Update sequence with predictions (rolling window)
            new_data = torch.from_numpy(day_pred).unsqueeze(0).to(device)
            current_sequence = torch.cat([current_sequence[1:], new_data], dim=0)
            current_weights = new_weights[0]
    
    # Convert to DataFrames
    pred_df = pd.DataFrame(
        np.array(pred_prices)[:, :, 1],  # Using Close prices for display
        index=dates,
        columns=assets
    )
    
    weight_df = pd.DataFrame(
        np.array(weight_history),
        index=dates,
        columns=assets
    )
    
    transaction_df = pd.DataFrame(
        np.array(transaction_history),
        index=dates,
        columns=assets
    )
    
    return pred_df, weight_df, transaction_df


# Example Usage
if __name__ == '__main__':
    # 1. Load your trained model
    model = PortfolioOptimization(num_assets=6, num_features=4, hidden_dim=128, input_seq_len=9).to(device)
    model.load_state_dict(torch.load("rl_portfolio_optimization.pth", map_location=device))
    
    # 2. Prepare initial data - replace with your actual last sequence
    seq_len, num_assets, num_features = 9, 6, 4
    initial_sequence = torch.randn(seq_len, num_assets, num_features).to(device)  # Replace with real data
    
    # 3. Set initial weights (must sum to 1)
    initial_weights = np.array([0.2, 0.15, 0.25, 0.1, 0.2, 0.1])  # Example weights
    
    # 4. Run prediction with reallocation
    pred_df, weight_df, transaction_df = predict_with_reallocation(
        model, 
        initial_sequence, 
        initial_weights,
        num_days=20,
        initial_capital=100000  # ₹1,00,000 initial investment
    )
    
    # 5. Save results
    with pd.ExcelWriter('portfolio_reallocation_results12.xlsx') as writer:
        pred_df.to_excel(writer, sheet_name='Predicted Prices')
        weight_df.to_excel(writer, sheet_name='Weight Allocation')
        transaction_df.to_excel(writer, sheet_name='Transactions')
    


In [31]:
if __name__ == "__main__":
    # Load model
    model = PortfolioOptimization(num_assets=6, num_features=4, hidden_dim=128, input_seq_len=9).to(device)
    model.load_state_dict(torch.load("rl_portfolio_optimization.pth", map_location=device))

    # Initial sequence: shape [10, 6, 4]
    initial_sequence = torch.randn(9, 6, 4)  # Replace with real data
    initial_weights = [0.2, 0.15, 0.25, 0.1, 0.2, 0.1]  # Must sum to 1

    pred_df, weight_df, transaction_df, value_df = predict_with_reallocation(
        model,
        initial_sequence=initial_sequence,
        initial_weights=initial_weights,
        num_days=10,
        initial_capital=10000
    )

    save_and_plot(pred_df, weight_df, transaction_df, value_df)


ValueError: not enough values to unpack (expected 4, got 3)

In [29]:
import pandas as pd
df=pd.read_excel("portfolio_reallocation_results12.xlsx")
df.head()

,Unnamed: 0,ONGC,BPCL,HINDPETRO,NMDC,IRFC,IOC
0,2025-07-20 23:04:42.670,1.226417,1.271512,1.289416,1.287446,1.285971,1.286717
1,2025-07-21 23:04:42.670,1.264654,1.308849,1.350248,1.333076,1.345595,1.332076
2,2025-07-22 23:04:42.670,1.258265,1.300650,1.335336,1.334207,1.337342,1.329698
3,2025-07-23 23:04:42.670,1.260016,1.296512,1.337728,1.343762,1.336334,1.327741
4,2025-07-24 23:04:42.670,1.248338,1.287453,1.324426,1.317788,1.321180,1.311705
